In [41]:
import torch
import torch.optim as optim

In [38]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0] # Celsius
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] # unknown
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [4]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples) # random permutation of ints from 0 to n_samples - 1
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
train_indices, val_indices

(tensor([ 2,  7,  8,  4,  1,  5,  3, 10,  9]), tensor([0, 6]))

In [49]:
train_t_u = t_u[train_indices].unsqueeze(1)
train_t_c = t_c[train_indices].unsqueeze(1)

val_t_u = t_u[val_indices].unsqueeze(1)
val_t_c = t_c[val_indices].unsqueeze(1)

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

## Artificial Neural Networks
Note: saturated when changes in input result in little/no changes to output.

PyTorch has building blocks called modules. A PyTorch module is a class derived from nn.Module and can have one or more Parameter instances as attributes, which are tensors whose value is optimized during the training process.

In [27]:
import torch.nn as nn

# Linear is a subclass of nn.Module, which means it has a __call__ method defined. Use it.
linear_model = nn.Linear(1, 1) # args are input size, output size, bias(=True default)
linear_model(val_t_un.unsqueeze(1))

tensor([[0.8432],
        [0.7940]], grad_fn=<AddmmBackward0>)

In [28]:
linear_model.weight

Parameter containing:
tensor([[0.2734]], requires_grad=True)

In [29]:
linear_model.bias

Parameter containing:
tensor([-0.1328], requires_grad=True)

In [32]:
x = torch.ones(1)
linear_model(x)

tensor([0.1406], grad_fn=<AddBackward0>)

Any module in ```nn``` written to produce output for a batch of multiple inputs. The first dimension of the input is the batch dim.

Need tensor of size B x Nin where B is size of batch and Nin is number of inputs. For example, for 10 samples need 10 x 1 input.

Output is tensor of size B x Nout.

In [33]:
x = torch.ones(10, 1)
linear_model(x)

tensor([[0.1406],
        [0.1406],
        [0.1406],
        [0.1406],
        [0.1406],
        [0.1406],
        [0.1406],
        [0.1406],
        [0.1406],
        [0.1406]], grad_fn=<AddmmBackward0>)

Change prev input to be of desired input form B x Nin.

In [39]:
t_c = t_c.unsqueeze(1)
t_u = t_u.unsqueeze(1)
t_c.shape, t_u.shape

(torch.Size([11, 1]), torch.Size([11, 1]))

Now fit model.

In [42]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

In [44]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[-0.8879]], requires_grad=True),
 Parameter containing:
 tensor([0.4475], requires_grad=True)]

Now create training loop with model passed in instead of individual params. Also use loss functions from ```torch.nn``` directly, which are subclasses of ```nn.Module``` so need to be created as an instance.

In [54]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        train_loss = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        val_loss = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

In [55]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(3000, optimizer, linear_model, nn.MSELoss(), train_t_un, val_t_un, train_t_c, val_t_c)

print(linear_model.weight, linear_model.bias)

Epoch 1, Training loss 161.5184, Validation loss 1.6687
Epoch 1000, Training loss 4.3727, Validation loss 4.4504
Epoch 2000, Training loss 2.9499, Validation loss 2.9953
Epoch 3000, Training loss 2.8603, Validation loss 3.2862
Parameter containing:
tensor([[5.4070]], requires_grad=True) Parameter containing:
tensor([-17.5867], requires_grad=True)
